In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

# 데이터 불러오기 / 데이터 정리

In [3]:
IT_finance = pd.read_excel('./data/dongboo/IT_TOTAL_재무사항.xlsx')
Medicine_finance = pd.read_excel('./data/dongboo/제약_TOTAL_재무사항.xlsx')

In [4]:
IT_stock = pd.read_excel('./data/dongboo/IT_TOTAL_주가.xlsx')
Medicine_stock = pd.read_excel('./data/dongboo/제약_TOTAL_주가.xlsx')

In [5]:
IT = pd.merge(IT_finance,IT_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])
Medicine = pd.merge(Medicine_finance,Medicine_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])

# 회사별로 분리

In [6]:
lg = IT[IT['회사명'] == 'LG이노텍(주)']
sdi = IT[IT['회사명'] == '삼성SDI(주)']
ssem = IT[IT['회사명'] == '삼성전기(주)']
hynix = IT[IT['회사명'] == '에스케이하이닉스(주)']

In [7]:
greencross = Medicine[Medicine['회사명'] == '(주)녹십자']
daewoong = Medicine[Medicine['회사명'] == '(주)대웅제약']
yoohan = Medicine[Medicine['회사명'] == '(주)유한양행']
hanmi = Medicine[Medicine['회사명'] == '한미약품(주)']

In [8]:
def classify(stock):
    
    # 회계년도 순으로 정렬 및 인덱스 초기화
    stock.sort_values('회계년도', ascending= True, inplace = True)
    
    stock.reset_index(drop = True, inplace = True)
    
    
    # 새로운 변수 추가
    
    stock['3개월후종가'] = stock['종가'][1:].reset_index(drop=True)
    
    stock['분기수익률'] = (stock['3개월후종가'] - stock['종가']) / stock['종가']
    
    stock['수익률상승하락'] = stock['분기수익률']
    
    for i in range(len(stock)):
        if stock['분기수익률'][i] < 0.005:
            stock['수익률상승하락'][i] = 0
        else:
            stock['수익률상승하락'][i] = 1
            
    return stock

In [9]:
classify(lg)
classify(sdi)
classify(ssem)
classify(hynix)

classify(greencross)
classify(daewoong)
classify(yoohan)
classify(hanmi)

,회사명,거래소코드,회계년도,자산(*),자본(*),* 수권주식수,*주당 액면가액,* 발행한 주식총수(*),보통주,우선주,지배기업주주지분(*),자본금(*),보통주자본금,우선주자본금,신종자본증권,자본잉여금(*),주식발행초과금,배당건설이자,주식매입선택권,감자차익,보험차익,합병차익,자산수증이익,채무면제이익,자기주식처분이익,전환권대가,신주인수권대가,교환권대가,전환권재매입이익,기타자본잉여금,재평가적립금(2001년 이전),기타자본(*),자기주식,자기주식처분손실,주식할인발행차금,주식매입선택권.1,감자차손,파생상품평가이익,파생상품평가손실,전환권재매입손실,기타,기타포괄손익누계액(*),금융자산평가손익,매도가능금융자산평가손익,관계기업 등 기타포괄손익,환산 관련 외환차이 적립금(환율변동차이),해외사업장순투자위험회피 적립금,확정급여제도의 보험수리적 손익 적립금,확정급여제도의 재측정요소,현금흐름위험회피 적립금,주식기준보상 적립금,재평가잉여금,기타.1,이익잉여금(결손금)(*),* (지배)당기순손익,이익준비금,재무구조개선적립금,기업합리화적립금,기타법정적립금,재평가손익,보험수리적손익,기타이익잉여금,미처분이익잉여금(결손금),*전기이월미처분이익잉여금(결손금),비지배주주지분,* (비지배)당기순손익,*비지배주주지분초과손실액,부채(*),* (정상)영업손익(보고서기재)(3개월),* (정상)영업손익(계산수치)(3개월),기타이익(*)(3개월),이자수익(3개월),배당금수익(3개월),외환거래이익(*)(3개월),외환차익(3개월),외환환산이익(3개월),기타외화거래이익(3개월),보험차익(3개월),퇴직연금운용이익(3개월),전기오류수정이익(3개월),기타(3개월),기타손실(*)(3개월),이자비용(3개월),외환거래손실(*)(3개월),외환차손(3개월),외화환산손실(3개월),기타외환거래손실(3개월),법인세추납액(3개월),퇴직연금운용손실(3개월),전기오류수정손실(3개월),기타(3개월).1,법인세비용차감전순이익(손실)(3개월),법인세비용(3개월),종속회사 매수일전 순손익(3개월),처분된 종속회사 순손익(3개월),계속영업이익(손실)(3개월),중단영업이익(손실)(3개월),*중단영업손익 법인세효과(3개월),당기순이익(손실)(3개월),기타포괄손익(*)(3개월),금융자산평가손익(3개월),매도가능금융자산평가손익(3개월),관계기업 등 기타포괄손익(3개월),환산 관련 외환차이(환율변동차이)(3개월),해외사업장순투자위험회피(3개월),현금흐름위험회피 적립금(3개월),확정급여제도의 보험수리적손익(3개월),확정급여제도의 재측정요소(3개월),주식기준보상 적립금(3개월),재평가손익(3개월),기타(3개월).2,기타포괄손익관련 법인세(3개월),총포괄손익(3개월),*(당기순손익 귀속)지배기업주주지분(3개월),*계속영업손익(3개월),*중단영업손익(3개월),*(당기순손익 귀속)비지배주주지분(3개월),*계속영업손익(3개월).1,*중단영업손익(3개월).1,*(총포괄손익 귀속)지배기업주주지분(3개월),*(총포괄손익 귀속)비지배주주지분(3개월),*주당계속영업이익(3개월),*주당순손익(3개월),*희석주당계속영업이익(3개월),*희석주당순손익(3개월),*(지배기업주주지분)주당계속영업이익(3개월),*(지배기업주주지분)주당순손익(3개월),*(지배기업주주지분)희석주당계속영업이익(3개월),*(지배기업주주지분)희석주당순손익(3개월),* 성격별 비용계정(*)(3개월),제품과 재공품의 변동(3개월),기업이 수행한 용역으로서 자본화되어 있는 부분(3개월),원재료와 저장품(소모품)의 사용액(3개월),상품의 판매(3개월),기타원가(3개월),종업원 급여비용(3개월),감가상각비와 기타상각비 및 손상차손(3개월),세금과공과(3개월),대손상각비(3개월),물류비(운송보관)(3개월),광고 및 판매촉진비(3개월),임차료 및 리스료(3개월),경상연구개발비(3개월),기타비용(3개월),기말자본금(*),기초자본금,유상증자(감자),무상증자(감자),주식매수선택권,전환사채,신주인수권부사채,주식배당,합병분할(영업양수도 등)로 인한 변동,회계정책변경누적효과,오류수정 등,연결범위의 변동,외화환산차이,기타.2,기말신종자본증권(*),기초신종자본증권,신종자본증권의 발행,기타.3,기말자본잉여금(*),기초자본잉여금,유상증자(감자).1,무상증자(감자).1,주식발행초과금.1,주식매수선택권.1,감자차손익,보험차익.1,자기주식처분손익,전환권대가(전환사채),신주인수권대가(신주인수권부사채),교환권대가(교환사채),전환권재매입손익,결손금처리,지분법자본잉여금,합병분할(영업양수도 등)로 인한 변동.1,회계정책변경누적효과.1,오류수정 등.1,연결범위의 변동.1,외화환산차이.1,기타.4,기말기타자본(*),기초기타자본,자기주식.1,자기주식처분손실.1,주식할인발행차금.1,주식매입선택권.2,파생상품평가이익.1,파생상품평가손실.1,출자전환채무,신주청약증거금,감자차손익.1,전환권재매입손익.1,지분법자본조정,합병분할(영업양수도 등)로 인한 변동.2,회계정책변경누적효과.2,오류수정 등.2,연결범위의 변동.2,외화환산차이.2,기타.5,기말기타포괄손익누계액(*),기초기타포괄손익누계액,금융자산평가손익.1,매도가능금융자산 평가손익,관계기업 등 기타포괄손익.1,환산 관련 외환차이 적립금(환율변동차이).1,해외사업장순투자위험회피 적립금.1,확정급여제도의 보험수리적 손익 적립금.1,확정급여제도의 재측정요소.1,현금흐름위험회피 적립금.1,주식기준보상 적립금.1,재평가잉여금.1,합병분할(영업양수도 등)로 인한 변동.3,회계정책변경누적효과.3,오류수정 등.3,연결범위의 변동.3,외화환산차이.3,기타.6,기말이익잉여금(결손금)(*),처분후 이익잉여금(결손금)(*),수정후 이익잉여금(결손금)(*),기초이익잉여금(결손금),회계정책변경누적효과.4,전기오류수정손익,기타.7,(연차배당),(신종자본증권 분배금),주식할인발행차금의 상각,자기주식처분손실의 상각,결손금처리.1,기타이익잉여금처분액,(중간배당),확정급여제도의 재측정요소.2,기타포괄손익,당기순이익(총포괄손익),보험수리적손익.1,재평가손익.1,지분법이익잉여금,합병분할(영업양수도 등)로 인한 변동.4,연결범위의 변동.4,외화환산차이.4,기타.8,기말비지배주주지분(*),처분후 비지배주주지분(*),수정후 비지배주주지분(*),기초 비지배주주지분,회계정책변경누적효과.5,전기오류수정손익.1,기타.9,(종속회사의 배당금),기타.10,(중간배당).1,당기순이익(총포괄손익).1,재평가잉여금.2,금융자산평가손익.2,매도가능금융자산 평가손익.1,관계기업 등 지분법평가손익,환산 관련 외환차이 적립금(환율변동차이).2,해외사업장순투자위험회피 적립금.2,확정급여제도의 보험수리적 손익 적립금.2,확정급여제도의 재측정요소.3,현금흐름위험회피 적립금.2,주식기준보상 적립금.2,기타포괄손익누계액,합병분할로 인한 변동,연결범위의 변동.5,외화환산차이.5,기타.11,합계,영업활동으로 인한 현금흐름(간접법)(*),*(직접법)총현금유입(*),재화의 판매와 용역제공에 따른 현금유입,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유입",이자수취액,배당금수취액,법인세환급,기타현금유입액,*(직접법)총현금유출(*),종업원과 관련하여 직·간접으로 발생하는 현금유출,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유출",이자비용유출액,배당금지급액,법인세납부,단기매매목적으로 보유하는 계약에서 발생하는 현금유출,기타현금유출액,당기순손익,법인세비용차감전순손익,현금 유출이 없는 비용 등 가산(*),전기오류수정손익.2,비지배주주지분순손익,보험수리적손실,기타.12,(현금 유입이 없는 수익 등 차감)(*),전기오류수정손익.3,비지배주주지분순손익.1,보험수리적이익,기타.13,영업활동으로 인한 자산부채 변동(운전자본 변동)(*),자산의 감소(증가)(*),부채의 증가(감소)(*),정부보조금 등의 변동,기타 운전자본의 변동,*영업에서 창출된 현금흐름,이자수입,이자지급(-),배당금수입,배당금지급(-),법인세환입,법인세납부(-),중단영업관련 현금흐름,투자활동으로 인한 현금흐름(*),투자활동으로 인한 현금유입액(*),정부보조금 등의 변동.1,종속기업 및 기타사업의 지배력 관련한 현금흐름,합병분할(양수도등)으로 인한 현금유입액,기타투자활동으로 인한 현금유입액,(투자활동으로 인한 현금유출액)(*),정부보조금 등의 변동.2,종속기업 및 기타사업의 지배력 관련한 현금흐름.1,

# 앞서 유한양행만 상관관계 분석: 변수 <3개월 후 종가>와의 상관관계 분석

In [11]:
df_yoohan = pd.DataFrame(yoohan.corr()['종가'].reset_index().sort_values('종가', ascending = False))
df_yoohan.columns = ['지표','종가']
df_yoohan

,지표,종가
376,종가,1.000000
373,최고가,0.990520
375,산술평균,0.986460
374,최저가,0.982653
65,부채(*),0.759637
379,3개월후종가,0.706961
225,(연차배당),0.653025
1,자산(*),0.638677
378,거래대금,0.631190
329,재무활동으로 인한 현금유입액(*),0.629439


# 산업별 변수 <3개월 후 종가>와의 상관관계 분석

In [ ]:
# 밑에 함수로 해보기!!!!

In [ ]:
nc_corr = nc.corr(method = 'pearson')
df_nc = pd.DataFrame(nc_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_nc.columns = ['지표','종가']

sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_sdi.columns = ['지표','종가']

ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_ssem.columns = ['지표','종가']

hynix_corr = hynix.corr(method = 'pearson')
df_hynix = pd.DataFrame(hynix_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_hynix.columns = ['지표','종가']

In [ ]:
list_corr = []
order = 15

for i in range(len(df_nc.index[:order])):
    list_corr.append(df_nc.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [ ]:
var = []
for i in range(len(variable)):
    var.append(df_nc['지표'][variable.index[i]])
var = pd.DataFrame(var)
var.columns = ['var']

In [ ]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(20)